-----

## Tournament sums

Burton Rosenberg

29 May 2023


----



The CPU sort of sum loop is $O(n)$. With parallelism it is possible to reduce this to $O(\log n)$ time, with linear threads by creating a tournement. This is possible with any associative operation. This consists in effect of replacing the parenthesization of, 

$$
S = (a_1+(a_2+(\ldots + a_n)))
$$

with any sort of index pattern such as,

$$
S = ((a_1+a_2)+(a_3+a_4))+((a_5+a_6)+(a_7+a_8))
$$

This pattern is easily extended out to $n=2^k$, and other values of $n$ are possible with a little care at the edge.

There are various patterns this can be done, each represented by a particular loop invariant.

### Warp considerations

NVidia launches threads in groups of 32, called a _warp_. This is done to collect up the
scheduling overhead. 

- Warps have consecutive thread numbers, 32 aligned.
- Warps run exactly that same code line by line.

Even though a warp runs the same code, line by line, branches and loops can take different paths in the code flow, called  _warp divergence_. Every thread in a warp runs the same code path, with the results disabled if in the logic of that particular thread, the branch should not be executed.

Various ways of arranging the tournament will resuilt in fewer threads being activated. In the folding variant given, since we always work with consecutive array locations, and the array locatins and thread ID's are identified, we keep all the threads in our warps working (until we are below 32).


### Thread synchronization considerations

The tournament must go in clear stages, with all threads finishing one folding before going on to the next. Since warps will proceed at their own pace, they must be explicitly synchronized at each $k$.

#### Thread synchronization on the GPU

There are synchronization calls that form a classic synchronization barrier, will all threads must arrive at the barrier before they all then proceed on. However, this synchronization only works in a thread block, and thread blocks are small.

#### Thread synchronization using streams

The call to launch a kernel, with its thread multiplicity is placed by the CPU in a GPU command stream and run asynchronously from the CPU. If the CPU attempts to launch several kernels one after the other, they will queue up on the GPU immediately returning to the CPU. 

However, one thread launch will not start until the thread launch proceeding it in the queue has exited. In this way all threads from one launch are completed before any thread in the next launch begins.

__Note:__ NVidia makes use of named for default streams. They act a bit differently with respect to synchronization. Also, the copy of data to and from the GPU is synchronized. This is a simple way for a program learns that a thread launch is completed. While the launches are queued immediately and the code in the CPU continues asynchronously, it will block waiting for the data copy of the result from the GPU to the CPU.

### Memory considerations

The NVidia has many classes of memory of which we shall briefly consider two: global and shared. In our examples here we are using global memory. This memory is accessible in a uniform way by all threads on the GPU.

Shared memory is shared between threads in a block, with a different shared memory for each block and no way for a thread to access memory from a block other than its own. Shared memory is also arranged in 32 banks, and reads and writes to each bank occur in parallel.

The global memory is simple, but it is not parallelized as are the threads. There will be consierable contention if all threads access different locations. We have ignored this issue and just pretended our memory is extremely fast and we pay no price for concurrent access. 


In [1]:
#
# summing elements in an array
# Accelerate! GPU course
#
# last-update:
#     30 aug 2022 -bjr: created
#
#

# this is a simulation of a GPU algorithm; 
# the phases are sequential, and should synchronize
# the threads are parallel 

# properties: 
#    threads used: n/2^i in phase i
#    memory accesss: fully independent

def total_sum_folding(a): 

    def above_power_2(x):
        i = 1
        while i<x:
            i *= 2
        return i

    l_a = above_power_2(len(a))
    
    def gpu_kernel(thread_idx,a, d):
        a[thread_idx] += a[thread_idx+d]
        
    phase = 0
    # the first fold takes care of arrays not a power of two
    d = l_a//2
    
    # thread launch
    for thread in range(d):
        if (thread+d)<len(a):
            gpu_kernel(thread,a, d)

    # remaining phasess do not need the length check
    d = d//2
    
    # Loop Invariant, 
    #    For i = 0, ... , n/2^k-1, cell i contains sum_{j<k} a[i+k*n/2^k]
    # Basis: 2^k==n, trivial
    # Update: fold the array
    # Final: cell 0 has the sum of all a[i]
    
    while d>0:
        phase += 1 # just a notation;
        
        # thread launch
        for thread in range(d):
            gpu_kernel(thread,a, d)
        d = d//2
            
    return


In [2]:
def total_sum_seq(a):
    s = 0
    for i in range(len(a)):
        s += a[i]
    a[0] = s
    return


def testing_total_sum(a,b):

    def fill_array(a):
        for i in range(len(a)):
            a[i] = i
        return a

    def testing_total_sum_aux(n):
        a = [0]*n
        
        fill_array(a)
        total_sum_seq(a)
        s_1 = a[0]
        
        fill_array(a)
        total_sum_folding(a)
        s_2 = a[0]
        
        if s_1!=s_2:
            return False
        return True

    print(f'Testing total sum between {a} and {b} ...')
    for s in range(a,b):
        if not testing_total_sum_aux(s):
            print('** error')
            return False
    print('** success')
    print()
    return True

testing_total_sum(1000,1050)

Testing total sum between 1000 and 1050 ...
** success



True

### Exercise

Create a different tournament pattern, for instance the one suggest at the beginning of first adding neighbors, then neighbors with lowest bit of the index zero, then neightbors with the two lowest bits of the index zero, etc. Remember to write a loop invariant.

To make things a bit different, calculate the maximum value in the array, rather than the sum of all numbers in the array.